In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
import pandas as pd

df = pd.read_csv('./spam mail.csv')
df.head()

,Category,Masseges
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.groupby('Category').describe()

Masseges                                                            \
            count unique                                                top   
Category                                                                      
ham          4825   4516                             Sorry, I'll call later   
spam          747    653  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [4]:
df['Category'].value_counts()

,count
Category,
ham,4825
spam,747


In [5]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [6]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [7]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [8]:
df_balanced = pd.concat([df_spam,df_ham_downsampled])
df_balanced.shape

(1494, 2)

In [9]:
df_balanced['Category'].value_counts()

,count
Category,
spam,747
ham,747


In [10]:
df_balanced.sample(5)

,Category,Masseges
2260,spam,SplashMobile: Choose from 1000s of gr8 tones e...
1107,ham,From someone not to smoke when every time I've...
3423,spam,Am new 2 club & dont fink we met yet Will B gr...
1064,ham,"That's fine, I'll bitch at you about it later ..."
4956,ham,I'm vivek:)i got call from your number.


In [11]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda X: 1 if X == 'spam' else 0)
df_balanced.sample(5)

,Category,Masseges,spam
5249,ham,"K I'm leaving soon, be there a little after 9",0
4700,ham,I liked the new mobile,0
1713,spam,Hard LIVE 121 chat just 60p/min. Choose your g...,1
5228,spam,PRIVATE! Your 2003 Account Statement for <fone...,1
3066,ham,Ok. So april. Cant wait,0


In [15]:
from sklearn.model_selection import train_test_split

X_trian,X_test,y_train,y_test = train_test_split(df_balanced['Masseges'],df_balanced['spam'],stratify=df_balanced['spam'])

In [16]:
X_trian.head(4)

,Masseges
3009,Imagine Life WITHOUT ME... see.. How fast u ar...
3893,Dear Dave this is your final notice to collect...
4953,U coming back 4 dinner rite? Dad ask me so i r...
3900,That depends. How would you like to be treated...


In [32]:
bert_preprocess = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3")
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-l-12-h-768-a-12/4",trainable = True)


In [33]:
def get_sentence_embeding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding(["harry is my name",
                       "Harry, are you OK>"])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.7844163 , -0.23808338,  0.46393085, ...,  0.2649192 ,
        -0.55504096,  0.8142713 ],
       [-0.856149  , -0.41788098, -0.9661119 , ..., -0.782106  ,
        -0.6539984 ,  0.89485127]], dtype=float32)>

In [34]:
e = get_sentence_embeding([
    'banana',
    'grapes',
    'apple',
    'cricket',
    'football',
    'i am the best'
])

In [35]:
e

<tf.Tensor: shape=(6, 768), dtype=float32, numpy=
array([[-0.760692  , -0.14219385,  0.49604586, ...,  0.42165312,
        -0.532214  ,  0.8031219 ],
       [-0.86023223, -0.2124296 ,  0.49156848, ...,  0.39797997,
        -0.60506296,  0.8447167 ],
       [-0.8196456 , -0.29609606,  0.20951797, ...,  0.25593394,
        -0.5874299 ,  0.8434556 ],
       [-0.92026347, -0.40818018, -0.26689315, ..., -0.19524567,
        -0.62910986,  0.8874004 ],
       [-0.9697425 , -0.5758805 , -0.92251605, ..., -0.6530098 ,
        -0.75273716,  0.952486  ],
       [-0.8588855 , -0.32033417,  0.37063643, ...,  0.20212623,
        -0.62839586,  0.899951  ]], dtype=float32)>

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[0]],[e[1]])
cosine_similarity([e[0]],[e[4]])
cosine_similarity([e[3]],[e[4]])



array([[0.92521024]], dtype=float32)

In [43]:
##bert layers
text_input = tf.keras.layers.Input(shape=(),dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
#Neural network layers

layer = tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
layer = tf.keras.layers.Dense(1,activation='sigmoid',name='output')(layer)
#construct final model
model = tf.keras.Model(inputs=[text_input],outputs=[1])

ValueError: Exception encountered when calling layer 'keras_layer_2' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_2' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=False, name=text>
  • training=None

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
#from official.nlp import optimization  # to create AdamW optimizer

# Load BERT preprocessing and encoder from TensorFlow Hub
bert_preprocess = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3")
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-l-12-h-768-a-12/4",trainable = True)

# Define the input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

# Preprocess the input text
preprocessed_text = bert_preprocess(text_input).get('input_word_ids')

# Encode the preprocessed text
outputs = bert_encoder(preprocessed_text)

# Add a dropout layer
dropout = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])

# Add a dense layer with softmax activation for classification
num_classes = 2  # Adjust this based on your specific use case
output = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(dropout)

# Construct the final model
model = tf.keras.Model(inputs=[text_input], outputs=[output])

# Compile the model with an optimizer, loss function, and evaluation metrics
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = tf.metrics.SparseCategoricalAccuracy()

# Set up the optimizer with a learning rate schedule
epochs = 3
steps_per_epoch = 1000  # Adjust based on your dataset
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1 * num_train_steps)

init_lr = 3e-5
#optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          #num_train_steps=num_train_steps,
                                          #num_warmup_steps=num_warmup_steps,
                                          #optimizer_type='adamw')

model.compile(optimizer='adam',
              loss=loss,
              metrics=[metrics])

# Display the model's architecture
model.summary()


2025-03-23 07:05:19.796327: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-23 07:05:20.031065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742692820.138122     531 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742692820.163154     531 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 07:05:20.367116: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

RuntimeError: Op type not registered 'CaseFoldUTF8' in binary running on archlinux. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.